<a href="https://colab.research.google.com/github/Sazim2019331087/voice_model/blob/main/voice_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing libraries

In [1]:
!pip install librosa numpy pandas scikit-learn

# Importing libraries

In [2]:
import librosa
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os

# Load CSV file with person's data

In [3]:
data = pd.read_csv('primary_data.csv')

# Check data samples in csv file

In [6]:
data = data.dropna()
data.head()

,email,name,age,gender,institute,dept_or_cls,language,audio_ffile
1,shadmim-cse@sust.edu,Sifat,27.0,Male,Teacher,Thakurgaon,Bangla,6759b4871227a.wav
2,codeanindya@gmail.com,Anindya,15.0,Male,Scholarshome,Class 9,English,675c793628169.wav
3,tahmidkabiraddin@gmail.com,Tahmid Kabir,24.0,Male,SUST,CSE,English,675d9552e3d4b.wav
4,pappu457373@gmail.com,Pappu Roy,24.0,Male,SUST,CSE,Bangla,6757a85d9e5e1.wav
5,joysourav37@gmail.com,Sourav Bhowmik Joy,25.0,Male,SUST,CSE,Bangla,6761c79890e95.wav


# Encoding email as numerical labels

In [8]:
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['email'])
data.head()

,email,name,age,gender,institute,dept_or_cls,language,audio_ffile,label
1,shadmim-cse@sust.edu,Sifat,27.0,Male,Teacher,Thakurgaon,Bangla,6759b4871227a.wav,11
2,codeanindya@gmail.com,Anindya,15.0,Male,Scholarshome,Class 9,English,675c793628169.wav,1
3,tahmidkabiraddin@gmail.com,Tahmid Kabir,24.0,Male,SUST,CSE,English,675d9552e3d4b.wav,13
4,pappu457373@gmail.com,Pappu Roy,24.0,Male,SUST,CSE,Bangla,6757a85d9e5e1.wav,5
5,joysourav37@gmail.com,Sourav Bhowmik Joy,25.0,Male,SUST,CSE,Bangla,6761c79890e95.wav,3


# Extracting MFCC Features

In [9]:
def extract_mfcc(file_path, n_mfcc=13, max_len=130):
    audio, sample_rate = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=n_mfcc)
    mfcc = np.pad(mfcc, ((0, 0), (0, max(0, max_len - mfcc.shape[1]))), mode='constant')
    return mfcc[:, :max_len]

# Data Preparation for model

In [11]:
X, y = [], []
for _, row in data.iterrows():
    file_path = os.path.join('voices', row['audio_ffile'])
    if os.path.exists(file_path):
        mfcc_features = extract_mfcc(file_path)
        X.append(mfcc_features)
        y.append(row['label'])

<ipython-input-9-4c1d2a8242f5>:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(file_path, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-9-4c1d2a8242f5>:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(file_path, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-9-4c1d2a8242f5>:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(file_path, sr=None)
/usr/local/lib/

# Data Shape for model

In [12]:
X = np.array(X)
y = np.array(y)

print("Features Shape:", X.shape)
print("Labels Shape:", y.shape)

Features Shape: (14, 13, 130)
Labels Shape: (14,)


# Sample features

In [15]:
X[0:2]

array([[[-6.3514685e+02, -6.3506653e+02, -6.3481854e+02, ...,
         -6.3184058e+02, -6.3268500e+02, -6.3506610e+02],
        [ 0.0000000e+00,  1.1288581e-01,  4.6094662e-01, ...,
          4.4878225e+00,  3.4037979e+00,  1.1132079e-01],
        [ 0.0000000e+00,  1.1081908e-01,  4.5103851e-01, ...,
          3.9650378e+00,  3.1796167e+00,  1.0292378e-01],
        ...,
        [ 0.0000000e+00,  5.1066417e-02,  1.7111459e-01, ...,
          1.7136666e-01,  6.8116337e-02, -4.5000918e-02],
        [ 0.0000000e+00,  3.9572645e-02,  1.1889296e-01, ...,
         -5.2137114e-02, -1.3146371e-01, -5.5218738e-02],
        [ 0.0000000e+00,  2.7597532e-02,  6.5137662e-02, ...,
         -3.6982465e-01, -2.7765200e-01, -6.0780726e-02]],

       [[-4.6928592e+02, -4.3035760e+02, -4.2865878e+02, ...,
         -3.2463712e+02, -3.2710052e+02, -3.3129858e+02],
        [ 8.0398819e+01,  9.6221138e+01,  8.4520340e+01, ...,
          1.6423160e+02,  1.5915305e+02,  1.5457370e+02],
        [ 3.7061352e+01, 

# Sample labels

In [17]:
y[0:5]

array([11,  1, 13,  5,  3])

# Model Development

## *Tensorflow installation for CNN*

In [18]:
!pip install tensorflow

## *Importing Tensorflow for CNN*

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

## *Build model*

In [20]:
model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(13, 130)),
    MaxPooling1D(pool_size=2),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## *Compiling Model*

In [21]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## *Train Model*

In [22]:
history = model.fit(X, y, epochs=50, batch_size=8, validation_split=0.2)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 340ms/step - accuracy: 0.0000e+00 - loss: 204.0208 - val_accuracy: 0.0000e+00 - val_loss: 41.5036
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.2045 - loss: 85.9403 - val_accuracy: 0.0000e+00 - val_loss: 27.5775
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.1629 - loss: 96.5967 - val_accuracy: 0.0000e+00 - val_loss: 45.3815
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.1212 - loss: 60.2932 - val_accuracy: 0.0000e+00 - val_loss: 61.5522
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.1023 - loss: 52.4582 - val_accuracy: 0.0000e+00 - val_loss: 67.0890
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.1212 - loss: 33.7594 - val_accuracy: 0.0000e+00 - val_loss: 56.4373
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.1023 - loss: 25.2990 - val_accuracy: 0.0000e+00 - val_loss: 45.3995
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.3674 - loss: 1

# Model Evaluation

In [24]:
loss, accuracy = model.evaluate(X, y)
print("Model Accuracy:", accuracy*100)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.7857 - loss: 3.8826
Model Accuracy: 78.57142686843872


# Speaker Recognition by the model

## *Prediction Function*

In [26]:
def predict_speaker(model, file_path):
    mfcc_features = extract_mfcc(file_path)
    mfcc_features = np.expand_dims(mfcc_features, axis=0)
    prediction = model.predict(mfcc_features)
    predicted_label = np.argmax(prediction)
    speaker_email = label_encoder.inverse_transform([predicted_label])
    return speaker_email[0]

## *Test Model by Prediction function*

In [34]:
test_file = '/content/testing_voices/67617f742f784.wav'
predicted_email = predict_speaker(model, test_file)
print("Predicted Speaker Email:", predicted_email)

<ipython-input-9-4c1d2a8242f5>:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(file_path, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted Speaker Email: codeanindya@gmail.com
